<b>Задание 1.</b> Проведите небольшое исследование алгоритма градиентного спуска. Оцените влияние значений скорости обучения (eta) и количества итераций на ошибку алгоритма. Как связаны эти два гиперпараметра между собой? Подберите скорость обучения и количество итераций до совпадения ответов алгоритма с результатами МНК. Как можно ускорить процесс вычисления весов?

In [36]:
import numpy as np
import math
import random

# Plotter:---------------------------------------------------------
import matplotlib.pyplot as plt

%matplotlib inline
from pylab import rcParams
rcParams['figure.figsize'] = 10,5
#--------------(plotter)-------------------------------------------

random.seed(42)
np.random.seed(42)




In [42]:

def calculate_uniform_a_b (mean=0,std_deviation=1) :
    c = 3 ** 0.5
    a = mean - c*std_deviation
    b = mean + c*std_deviation
    return a,b

def make_random_vector (mean=0,std_deviation=1,coordinate_number = 2, is_normal=True) :
    if (is_normal):
        return np.random.normal(mean,std_deviation,size=(coordinate_number))
    else :
        a,b = calculate_uniform_a_b(mean,std_deviation)
        return np.random.uniform(a,b,size=(coordinate_number))
    
def make_random_matrix (mean=0,std_deviation=1,row_number = 2, column_number = 2, is_normal=True) :
    if (is_normal):
        return np.random.normal(mean,std_deviation,size=(row_number,column_number))
    else :
        a,b = calculate_uniform_a_b(mean,std_deviation)
        return np.random.uniform(a,b,size=(row_number,column_number))
    
def make_linear_correlated_data (mean,std_deviation,coordinate_number,data_object_number) :
    
    a,b = calculate_uniform_a_b(mean,std_deviation)
    free_coef = random.uniform(a,b)
    
    weights = make_random_vector(mean,std_deviation,coordinate_number,is_normal=False)
    vectors = make_random_matrix(mean,std_deviation,data_object_number,coordinate_number,is_normal=True)
    weighted_vectors = vectors.dot(weights)+free_coef;
    noised_weighted_vectors = weighted_vectors + make_random_vector(mean,std_deviation/5,data_object_number,is_normal=True)
    
    return noised_weighted_vectors,vectors,weights,free_coef;
    

#nvs,vs,ws,fc = make_linear_correlated_data(0,3,2,10)
#ws


array([4.12798725, 4.16126652])

In [48]:
class LinearFunction :
    def __init__ (self,weights,threshold) :
        self.weights = weights
        self.threshold = threshold
    
    def calculate (self,x):
        return np.dot(x,self.weights)+self.threshold
    
    def correct(self,dw,etta,dt):
        delta = dw*etta
        self.weights+=delta
        delta = dt*etta
        self.threshold+=delta
        return self
    
class LinearRegression :
    
    def __init__ (self):
        self.linear_function = None;
    
    
    def fit (X,Y):
        
    def initialize (self,X):
        max = np.amax(X);
        min = np.amin(X);
        amp = (max-min)/10;
        self.linear_function = LinearFunction(make)
        
    
#lf = LinearFunction([1,0],-1);
#lf.calculate([[1,0],[0,1]])